# Notebook to create a pedestrian model for the narrow corridor
Run the two cells below only if you are on Google Colab. You will need to restart the session after running the first cell.

In [ ]:
!git clone https://github.com/c-pouw/physics-based-pedestrian-modeling.git
!pip install -e physics-based-pedestrian-modeling/

Afer restarting the session you can continue from the next cell.

In [ ]:
import os
os.chdir('physics-based-pedestrian-modeling/usage_notebooks')

If you run the notebook locally and you have installed the package you can start from this cell.

In [ ]:
from pathlib import Path
import os
import logging
from pprint import pformat

import numpy as np
import pandas as pd
from hydra import compose, initialize
import matplotlib.pyplot as plt
from omegaconf import OmegaConf

from physped.omegaconf_resolvers import register_new_resolvers
from physped.io.readers import trajectory_reader
from physped.io.writers import save_piecewise_potential
from physped.preprocessing.trajectories import preprocess_trajectories, process_slow_modes
from physped.core.functions_to_discretize_grid import learn_potential_from_trajectories
from physped.core.trajectory_simulator import simulate_trajectories
from physped.core.functions_to_select_grid_piece import evaluate_selection_point, evaluate_selection_range, get_index_of_the_enclosing_bin
from physped.visualization.plot_trajectories import plot_trajectories
from physped.visualization.plot_discrete_grid import plot_discrete_grid
from physped.visualization.plot_histograms import create_all_histograms, plot_multiple_histograms
from physped.core.piecewise_potential import PiecewisePotential
from physped.core.functions_to_discretize_grid import digitize_trajectories_to_grid

#### Read the configuration file

In [ ]:
# Set the environment name
env_name = "single_paths"

# Set the data source. Use 4tu to read the trajectories from a remote repository.
data_source = "4tu" 

with initialize(version_base=None, config_path="../physped/conf", job_name="test_app"):
    config = compose(
        config_name="config",
        return_hydra_config=True,
        overrides=[
            f"params={env_name}", 
            "params.data_source={data_source}",
            ],
    )
    register_new_resolvers(replace=True)

# set log level
log = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO, force = True)

# set plot style
plt.style.use(Path.cwd().parent / "physped/conf/science_no_latex.mplstyle")

# change working directory
working_dir = config.hydra.run.dir
os.makedirs(working_dir, exist_ok=True)
os.chdir(working_dir)

# log the model parameters
logging.info("MODELING PARAMETERS: \n" + pformat(OmegaConf.to_container(config.params.model, resolve=True), depth=1))

#### Read the raw trajectories from the remote repository on 4TU.

In [ ]:
trajectories = trajectory_reader[env_name](config)
logging.info('\n' + pformat(trajectories.head()))

#### Preprocess the trajectories.

In [ ]:
preprocessed_trajectories = preprocess_trajectories(trajectories, config=config)

#### Compute the slow dynamics.

In [ ]:
preprocessed_trajectories = process_slow_modes(preprocessed_trajectories, config)

#### Plot the preprocessed trajectories

In [ ]:
plot_trajectories(preprocessed_trajectories, config, "recorded")

#### Compute the piecewise potential

In [ ]:
piecewise_potential = learn_potential_from_trajectories(preprocessed_trajectories, config)

#### Simulate new trajectories with the model

In [ ]:
config.params.input_ntrajs = len(preprocessed_trajectories.Pid.unique())
simulated_trajectories = simulate_trajectories(piecewise_potential, config)

#### Plot the simulated trajectories

In [ ]:
plot_trajectories(simulated_trajectories, config, "simulated")